In [1]:
import cython
import numpy as np
import TidalPy
print(TidalPy.__version__)

0.5.0a5.dev1


In [2]:
%load_ext cython

In [3]:
%%cython --annotate --force
# distutils: language = c++

from libcpp cimport bool as cpp_bool_t
import numpy as np
cimport numpy as np

from scipy.constants import G as G_
from TidalPy.radial_solver.nondimensional import non_dimensionalize_physicals, re_dimensionalize_radial_func
from TidalPy.radial_solver.numerical.collapse import collapse_solutions
from TidalPy.radial_solver.numerical.initial import find_initial_guess
from TidalPy.radial_solver.numerical.interfaces import find_interface_func

# Import cythonized functions
from TidalPy.radial_solver.numerical.interfaces.interfaces_x cimport find_solution_num


# Setup globals
cdef double G
G = G_


def radial_solver_x(double[:] radius_array, double complex[:] complex_shear_array, double[:] bulk_array,
                    double[:] density_array, double[:] gravity_array, double frequency, double planet_bulk_density,
                    tuple is_solid_by_layer, tuple is_static_by_layer, tuple is_incompressible_by_layer,
                    tuple indices_by_layer,
                    unsigned int degree_l = 2, 
                    double complex[:] surface_boundary_conditions = None,
                    cpp_bool_t solve_load_numbers = False,
                    cpp_bool_t use_kamata = False,
                    int integration_method = 1,
                    double integration_rtol = 1.0e-8,
                    double integration_atol = 1.0e-9, 
                    cpp_bool_t verbose = False,
                    cpp_bool_t nondimensionalize = True,
                    ):
    # General indexing
    cdef Py_ssize_t i
    
    # Pull out key information
    cdef double radius_planet
    cdef Py_ssize_t num_layers, num_interfaces, num_radius
    radius_planet  = radius_array[-1]
    # TODO: Exception to check if there is at least one layer?
    num_layers     = len(is_solid_by_layer)
    num_interfaces = num_layers - 1
    num_radius     = len(radius_array)
    
    # Non-dimensionalize inputs
    cdef double G_to_use
    
    if nondimensionalize:
        radius_array, gravity_array, density_array, complex_shear_array, bulk_array, frequency, G_to_use = \
            non_dimensionalize_physicals(
                    radius_array, gravity_array, density_array, complex_shear_array, bulk_array, frequency,
                    mean_radius=radius_planet, bulk_density=planet_bulk_density
                    )
    else:
        G_to_use = G
    
    # Find boundary condition at the top of the planet -- this is dependent on the forcing type.
    #     Tides (default here) follow the (y2, y4, y6) = (0, 0, (2l+1)/R) rule
    cdef np.ndarray[np.complex128_t, ndim=1] bc_array
    cdef double complex[:] bc_array_view
    bc_array = np.empty(3, dtype=np.complex128)
    bc_array_view = bc_array
    
    if surface_boundary_conditions is None:
        # Assume tides
        for i in range(3):
            if i == 2:
                if nondimensionalize:
                    bc_array_view[i] == (2. * degree_l + 1.) / 1.
                else:
                    bc_array_view[i] == (2. * degree_l + 1.) / radius_planet
    else:
        # Use user input
        # TODO: Exception if user input is wrong size.
        for i in range(3):
            bc_array_view[i] = surface_boundary_conditions[i]
    
    # Find number of solutions per layer
    cdef np.ndarray[np.uint16_t, ndim=1] num_solutions_by_layer_array
    cdef unsigned short num_solutions_by_layer_view
    num_solutions_by_layer_array = np.zeros(num_layers, dtype=np.ushort)
    num_solutions_by_layer_view = num_solutions_by_layer_array
    
    cdef cpp_bool_t layer_is_solid, layer_is_static, layer_is_incomp
    
    for i in range(num_layers):
        
        layer_is_solid = is_solid_by_layer[i]
        layer_is_static = is_static_by_layer[i]
        layer_is_incomp = is_incompressible_by_layer[i]
        
        num_solutions_by_layer_array[i] = find_solution_num(layer_is_solid, layer_is_static, layer_is_incomp)
        
    
    # Store solutions per layer.
    # TODO: Make this a ndarray?
    cdef list solutions_by_layer_list
    solutions_by_layer_list = list()
    
    # Other variables
    cdef np.ndarray[np.complex128_t, ndim=2] solutions_below_array
    cdef double complex[:, :] solutions_below_view
    
    # Loop through layers from bottom to top and solve the differential equations
    for i in range(num_layers):
        
        # Find initial conditions at bottom of layer
        if i == 0:
            # Use initial condition function
            pass
        else:
            # Use results at the top of the layer below and a interface function
            pass
            
            
            
    
            
            
        
    

Content of stdout:
_cython_magic_3d9ec19b78e3d38e379328e5d5a5cdc3e4f0e866.cpp
C:\ProgramData\Anaconda3\envs\cytest\lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\joepr\.ipython\cython\_cython_magic_3d9ec19b78e3d38e379328e5d5a5cdc3e4f0e866.cpp(26308): warning C4551: function call missing argument list
   Creating library C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_3d9ec19b78e3d38e379328e5d5a5cdc3e4f0e866.cp310-win_amd64.lib and object C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_3d9ec19b78e3d38e379328e5d5a5cdc3e4f0e866.cp310-win_amd64.exp
Generating code
Finished generating code

ModuleNotFoundError: No module named 'TidalPy.radial_solver.numerical.interfaces.interfaces_x'

In [ ]:
%%cython --annotate --force
# distutils: language = c++
# distutils: language = c++
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False
""" Functions to calculate the initial conditions for an overlying liquid layer above another liquid layer.

For liquid-liquid layer interfaces: all the radial functions are continuous expect for if you are moving
from a dynamic layer to static layer or vice versa.

References
----------
S74   : Saito (1974; J. Phy. Earth; DOI: 10.4294/jpe1952.22.123)
TS72  : Takeuchi, H., and M. Saito (1972), Seismic surface waves, Methods Comput. Phys., 11, 217–295.
"""

import cython
from scipy.constants import G as G_

from libc.math cimport pi
from libcpp cimport bool as bool_cpp_t

cdef double G
G = G_

cpdef unsigned short find_solution_num(bool_cpp_t is_solid, bool_cpp_t is_static, bool_cpp_t is_compressible) nogil:
    """ Determine number of solutions required for layer based on assumptions.
    
    Parameters
    ----------
    is_solid : bool
        Layer is solid (True) or liquid (False).
    is_static : bool
        Use static (True) or dynamic (False) assumption.
    is_compressible : bool
        Use compressible (True) or incompressible (False) assumption.

    Returns
    -------
    num_sols : int
        Number of solutions required for layer.

    """

    # Initialize
    cdef unsigned short num_sols
    num_sols = 0

    if is_solid:
        if is_static:
            if is_compressible:
                num_sols = 3
            else:
                # TODO: Confirm
                num_sols = 3
        else:
            # Dynamic
            if is_compressible:
                num_sols = 3
            else:
                # TODO: Confirm
                num_sols = 3
    else:
        # Liquid
        if is_static:
            if is_compressible:
                num_sols = 1
            else:
                # TODO: Confirm
                num_sols = 1
        else:
            # Dynamic
            if is_compressible:
                num_sols = 2
            else:
                # TODO: Confirm
                num_sols = 2
    return num_sols


cpdef void interface_x(
        double complex[:, :] lower_layer_y, double complex[:, :] upper_layer_y,
        bool_cpp_t lower_is_solid, bool_cpp_t lower_is_static, bool_cpp_t lower_is_compressible,
        bool_cpp_t upper_is_solid, bool_cpp_t upper_is_static, bool_cpp_t upper_is_compressible,
        double interface_gravity, double liquid_density, double G_to_use = G
        ) nogil:

    cdef Py_ssize_t yi_lower, yi_upper, soli_lower, soli_upper
    cdef Py_ssize_t num_sols_lower, num_sols_upper, num_y_lower, num_y_upper
    num_sols_lower = len(lower_layer_y[0, :])
    num_sols_upper = len(upper_layer_y[0, :])
    num_y_lower    = len(lower_layer_y[:, 0])
    num_y_upper    = len(upper_layer_y[:, 0])

    cdef bool_cpp_t solid_solid, solid_liquid, liquid_solid, liquid_liquid
    solid_solid   = lower_is_solid and upper_is_solid
    solid_liquid  = lower_is_solid and not upper_is_solid
    liquid_solid  = not lower_is_solid and upper_is_solid
    liquid_liquid = not lower_is_solid and not upper_is_solid

    cdef bool_cpp_t static_static, static_dynamic, dynamic_static, dynamic_dynamic
    static_static   = lower_is_static and upper_is_static
    static_dynamic  = lower_is_static and not upper_is_static
    dynamic_static  = not lower_is_static and upper_is_static
    dynamic_dynamic = not lower_is_static and not upper_is_static

    cdef bool_cpp_t compress_compress, compress_incompress, incompress_compress, incompress_incompress
    compress_compress     = lower_is_compressible and upper_is_compressible
    compress_incompress   = lower_is_compressible and not upper_is_compressible
    incompress_compress   = not lower_is_compressible and upper_is_compressible
    incompress_incompress = not lower_is_compressible and not upper_is_compressible
    # TODO: Compressibility is not currently taken into account. This needs to be checked asap!

    # Other constants that may be needed
    cdef double complex lambda_1, lambda_2, coeff_1, coeff_2, coeff_3, coeff_4, coeff_5, coeff_6, frac_1, frac_2
    cdef double complex const_1, g_const

    g_const = 4. * pi * G_to_use

    if solid_solid:
        if static_static or static_dynamic or dynamic_static or dynamic_dynamic:
            # Does not matter if the layers are static or dynamic, solid-solid exchange perfectly.
            # All solutions carry over
            for yi_lower in range(num_y_lower):
                # They have same number of ys
                yi_upper = yi_lower
                for soli_lower in range(num_sols_lower):
                    # They have same number of sols
                    soli_upper = soli_lower
                    upper_layer_y[soli_upper, yi_upper] = lower_layer_y[soli_lower, yi_lower]
    elif liquid_liquid:
        # Liquid interfaces carry everything over if they are both static or both dynamic.
        if static_static or dynamic_dynamic:
            # All solutions carry over
            for yi_lower in range(num_y_lower):
                # They have same number of ys
                yi_upper = yi_lower
                for soli_lower in range(num_sols_lower):
                    # They have same number of sols
                    soli_upper = soli_lower
                    upper_layer_y[soli_upper, yi_upper] = lower_layer_y[soli_lower, yi_lower]
        elif static_dynamic:
            # For an upper layer that is liquid and dynamic there will be two independent solutions that need an initial guess.
            # # Solution 1
            # y_1_dynamic = 0
            upper_layer_y[0, 0] = 0. + 0.j
            # y_2_dynamic = -rho * y_5_static
            upper_layer_y[0, 1] = -liquid_density * lower_layer_y[0, 0]
            # y_5_dynamic = y_5_static
            upper_layer_y[0, 2] = lower_layer_y[0, 0]
            # y_6_dynamic = y_7_static + (4 pi G rho / g) * y_5_static
            upper_layer_y[0, 3] = (
                    lower_layer_y[0, 1] + (g_const * liquid_density / interface_gravity) * lower_layer_y[0, 0])
            # # Solution 2
            # y_1_dynamic = 1.
            upper_layer_y[1, 0] = 1. + 0.j
            # y_2_dynamic = rho * g * y_1_dynamic
            upper_layer_y[1, 1] = liquid_density * interface_gravity * upper_layer_y[1, 0]
            # y_5_dynamic = 0.
            upper_layer_y[1, 2] = 0. + 0.j
            # y_6_dynamic = -4 pi G rho y_1_dynamic
            upper_layer_y[1, 3] = -g_const * liquid_density * upper_layer_y[1, 0]
        elif dynamic_static:
            # lambda_j = (y_2j - rho * ( g * y_1j - y_5j))
            lambda_1 = (lower_layer_y[0, 1] -
                        liquid_density * (interface_gravity * lower_layer_y[0, 0] - lower_layer_y[0, 2]))
            lambda_2 = (lower_layer_y[1, 1] -
                        liquid_density * (interface_gravity * lower_layer_y[1, 0] - lower_layer_y[1, 2]))

            # Set the first coefficient to 1. It will be solved for later on during the collapse phase.
            coeff_1 = 1. + 0.j
            # The other coefficient which is related to 1 via...
            coeff_2 = -(lambda_1 / lambda_2) * coeff_1

            coeff_3 = lower_layer_y[0, 3] + g_const * lower_layer_y[0, 1] / interface_gravity
            coeff_4 = lower_layer_y[1, 3] + g_const * lower_layer_y[1, 1] / interface_gravity

            # y^liq(st)_5 = C^liq(dy)_1 * y^liq(dy)_5,1 + C^liq(dy)_2 * y^liq(dy)_5,2
            upper_layer_y[0, 0] = coeff_1 * lower_layer_y[0, 2] + coeff_2 * lower_layer_y[1, 2]
            # y^liq(st)_7 = C^liq(dy)_1 * y^liq(dy)_7,1 + C^liq(dy)_2 * y^liq(dy)_7,2
            upper_layer_y[0, 1] = coeff_1 * coeff_3 + coeff_2 * coeff_4
        else:
            # How did you get here...
            pass
    elif liquid_solid:
        if dynamic_dynamic or dynamic_static:
            # As far as I am aware, the dynamic_static and dynamic_dynamic solutions are the same.

            # See Eqs. 148-149 in TS72
            # For a dynamic solid layer there will be three independent solutions that we need an initial guess for.
            for soli_upper in range(3):
                if soli_upper == 0 or soli_upper == 1:
                    # For a dynamic liquid layer there will be two independent solutions at the top of the layer
                    upper_layer_y[soli_upper, 0] = lower_layer_y[soli_upper, 0]
                    upper_layer_y[soli_upper, 1] = lower_layer_y[soli_upper, 1]
                    upper_layer_y[soli_upper, 4] = lower_layer_y[soli_upper, 2]
                    upper_layer_y[soli_upper, 5] = lower_layer_y[soli_upper, 3]

                    # For solutions 1 and 2: y_3 and y_4 for the solid layer are zero
                    upper_layer_y[soli_upper, 2] = 0. + 0.j
                    upper_layer_y[soli_upper, 3] = 0. + 0.j
                else:
                    # For the third solid solution all the y's are set to zero except y_3.
                    upper_layer_y[soli_upper, 0] = 0. + 0.j
                    upper_layer_y[soli_upper, 1] = 0. + 0.j
                    upper_layer_y[soli_upper, 2] = 1. + 0.j
                    upper_layer_y[soli_upper, 3] = 0. + 0.j
                    upper_layer_y[soli_upper, 4] = 0. + 0.j
                    upper_layer_y[soli_upper, 5] = 0. + 0.j
        elif static_dynamic or static_static:
            # As far as I am aware, the static_static and static_dynamic solutions are the same.

            # Eqs. 20 in S74
            # For a static liquid layer there will be one independent solutions at the top of the layer
            upper_layer_y[0, 0] = 0. + 0.j
            # y_2_sol = -rho * y_5_liq
            upper_layer_y[0, 1] = -liquid_density * lower_layer_y[0, 0]
            upper_layer_y[0, 2] = 0. + 0.j
            upper_layer_y[0, 3] = 0. + 0.j
            # y_5_sol = y_5_liq
            upper_layer_y[0, 4] = lower_layer_y[0, 0]
            # y_6_sol = y_7_liq + (4 pi G rho / g) y_5_liq
            upper_layer_y[0, 5] = (lower_layer_y[0, 1] +
                                    (g_const * liquid_density / interface_gravity) * lower_layer_y[0, 0])
            # y_1_sol = 1.
            upper_layer_y[1, 0] = 1. + 0.j
            # y_2_sol = rho * g * y_1_sol
            upper_layer_y[1, 1] = liquid_density * interface_gravity * upper_layer_y[1, 0]
            upper_layer_y[1, 2] = 0. + 0.j
            upper_layer_y[1, 3] = 0. + 0.j
            upper_layer_y[1, 4] = 0. + 0.j
            # y_6_sol = -4 pi G rho y_1_sol
            upper_layer_y[1, 5] = -g_const * liquid_density * upper_layer_y[1, 0]

            upper_layer_y[2, 0] = 0. + 0.j
            upper_layer_y[2, 1] = 0. + 0.j
            # y_3_sol = 1.
            upper_layer_y[2, 2] = 1. + 0.j
            upper_layer_y[2, 3] = 0. + 0.j
            upper_layer_y[2, 4] = 0. + 0.j
            upper_layer_y[2, 5] = 0. + 0.j
    elif solid_liquid:

        if dynamic_dynamic or static_dynamic:
            # As far as I am aware, static_dynamic and dynamic_dynamic should be the same.
            # Eqs. 140-144 in TS72
            # Three solutions in the lower solid layer and two in the upper liquid.

            for soli_upper in range(2):
                # Solve for y^liq_1, y^liq_2, y^liq_5, y^liq_6 (TS72 Eq. 143)
                #    Note that the liquid solution does not have y_3, y_4 which are index 2, 3 for solid solution.
                coeff_1 = lower_layer_y[soli_upper, 3] / lower_layer_y[2, 3]
                upper_layer_y[soli_upper, 0] = lower_layer_y[soli_upper, 0] - coeff_1 * lower_layer_y[2, 0]
                upper_layer_y[soli_upper, 1] = lower_layer_y[soli_upper, 1] - coeff_1 * lower_layer_y[2, 1]
                upper_layer_y[soli_upper, 2] = lower_layer_y[soli_upper, 4] - coeff_1 * lower_layer_y[2, 4]
                upper_layer_y[soli_upper, 3] = lower_layer_y[soli_upper, 5] - coeff_1 * lower_layer_y[2, 5]
        elif dynamic_static or static_static:
            # As far as I am aware, static_static and dynamic_static should work the same.
            # Eq. 21 in S74
            frac_1 = -lower_layer_y[0, 3] / lower_layer_y[2, 3]
            frac_2 = -lower_layer_y[1, 3] / lower_layer_y[2, 3]

            # lambda_j = (y_2j + f_j y_23) - rho( g(y_1j + f_j y_13) - (y_5j + f_j y_53))
            lambda_1 = (lower_layer_y[0, 1] + frac_1 * lower_layer_y[2, 1]) - \
                        liquid_density * (interface_gravity * (lower_layer_y[0, 0] + frac_1 * lower_layer_y[2, 0]) -
                                        (lower_layer_y[0, 4] + frac_1 * lower_layer_y[2, 4]))
            lambda_2 = (lower_layer_y[1, 1] + frac_2 * lower_layer_y[2, 1]) - \
                        liquid_density * (interface_gravity * (lower_layer_y[1, 0] + frac_2 * lower_layer_y[2, 0]) -
                                        (lower_layer_y[1, 4] + frac_2 * lower_layer_y[2, 4]))

            # Set the first coefficient to 1. It will be solved for later on during the collapse phase.
            coeff_1 = 1.
            # The other two coefficients are related to 1 via...
            coeff_2 = -(lambda_1 / lambda_2) * coeff_1
            coeff_3 = frac_1 * coeff_1 + frac_2 * coeff_2

            const_1 = (g_const / interface_gravity)

            coeff_4 = lower_layer_y[0, 5] + const_1 * lower_layer_y[0, 1]
            coeff_5 = lower_layer_y[1, 5] + const_1 * lower_layer_y[1, 1]
            coeff_6 = lower_layer_y[2, 5] + const_1 * lower_layer_y[2, 1]

            # y^liq_5 = C^sol_1 * y^sol_5,1 + C^sol_2 * y^sol_5,2 + C^sol_3 * y^sol_5,3
            lower_layer_y[0, 1] = coeff_1 * lower_layer_y[0, 4] + coeff_2 * lower_layer_y[1, 4] + coeff_3 * lower_layer_y[2, 4]
            # y^liq_7 = C^sol_1 * y^sol_7,1 + C^sol_2 * y^sol_7,2 + C^sol_3 * y^sol_7,3
            lower_layer_y[0, 1] = coeff_1 * coeff_4 + coeff_2 * coeff_5 + coeff_3 * coeff_6


Content of stdout:
_cython_magic_072aa299743495bfb4f048e10c84b34ab3af3bfc.cpp
C:\Users\joepr\.ipython\cython\_cython_magic_072aa299743495bfb4f048e10c84b34ab3af3bfc.cpp(24574): warning C4551: function call missing argument list
   Creating library C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_072aa299743495bfb4f048e10c84b34ab3af3bfc.cp310-win_amd64.lib and object C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_072aa299743495bfb4f048e10c84b34ab3af3bfc.cp310-win_amd64.exp
Generating code
Finished generating code

In [19]:
%%cython --annotate --force
# distutils: language = c++
from libcpp cimport bool as cpp_bool_t
cimport numpy as np


def f(np.uint8_t[:] x):
    cdef int len_x, i
    len_x = len(x)
    
    for i in range(len_x):
        if x[i]:
            print('Yes!')
        else:
            print('No :(')

Content of stdout:
_cython_magic_043318e08c951a8d23cbf784613d282d2dd82ff9.cpp
C:\Users\joepr\anaconda3\envs\cytest310\lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\joepr\.ipython\cython\_cython_magic_043318e08c951a8d23cbf784613d282d2dd82ff9.cpp(18784): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
C:\Users\joepr\.ipython\cython\_cython_magic_043318e08c951a8d23cbf784613d282d2dd82ff9.cpp(24004): warning C4551: function call missing argument list
   Creating library C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_043318e08c951a8d23cbf784613d282d2dd82ff9.cp310-win_amd64.lib and object C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_043318e08c951a8d23cbf784613d282d2dd82ff9.cp310-win_amd64.exp
Generating code
Finished generating code

In [23]:
x = np.linspace(0., 3., 4)
y = x <= 2.
print(x, y)
f(y)

[0.000e+00 1.000e+00 2.000e+00 3.000e+00] [ True  True  True False]
Yes!
Yes!
Yes!
No :(
